Osnabrück University - Computer Vision (Winter Term 2020/21) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Axel Schaffland, Ludwig Schallner, Artem Petrov

# Exercise Sheet 07: Fourier Transform

## Introduction

This week's sheet should be solved and handed in before the end of **Saturday, January 02, 2021**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

**HINT: There is a bonus exercise in the end. If you do this bonus exercise you may leave out one of the other exercises.**

## Assignment 0: Math recap (Expectation and Variance) [0 Points] 

This exercise is supposed to be very easy, does not give any points, and is voluntary. There will be a similar exercise on every sheet. It is intended to revise some basic mathematical notions that are assumed throughout this class and to allow you to check if you are comfortable with them. Usually you should have no problem to answer these questions offhand, but if you feel unsure, this is a good time to look them up again. You are always welcome to discuss questions with the tutors or in the practice session. Also, if you have a (math) topic you would like to recap, please let us know.

**a)** What is the relation between mean and expectated value? How to compute it?

YOUR ANSWER HERE

**b)** What is the variance? What does it express? Why is there a square in the formula?

YOUR ANSWER HERE

**c)** Can you compute mean and variance of a given 1-dimensional dataset (e.g., $D=\{9,10,11,7,13\}$). Can you do the same for a 3-dimensional dataset (e.g., D=\{(1,10,9), (1,10,10), (10,10,11), (19,10,7), (19,10,13)\})?

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE

## Exercise 1: Understanding Fourier Transform [7 points]

This exercise aims at getting some intuition of finite, 2d-Fourier transform.

*Hint:* Python and numpy can deal with complex numbers: `np.real()` and `np.imag()` provide the real and imaginary parts. `np.abs()` and `np.angle()` provide amplitude and phase. `np.conj()` gives the complex conjugate.

**a)** Transform the image `dolly.png` into the frequency space (you may use the function
`numpy.fft.fft2`). The result will be a complex matrix. Plot histograms for the amplitude and phase
values. You may take the logarithm of the amplitude to enhance contrast.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('images/dolly.png')

# YOUR CODE HERE

def plot_hist(data, bar_width):
    # heights --> #elements in the bins
    # bins    --> value ranges for bins
    heights, bins = np.histogram(data, bins=50)
    # bins has one additional element --> [:-1]
    plt.bar(bins[:-1], heights, width=bar_width)

# computes the n-dimensional discrete Fourier transform
ft = np.fft.fft2(img)
amplitude = np.abs(ft)
log_amplitude = np.log(amplitude)
phase = np.angle(ft)

# generate plots
plt.figure(figsize=(15, 10))
plt.gray()

plt.subplot(2, 3, 1); plt.title('original img'); plt.imshow(img)
plt.subplot(2, 3, 2); plt.title('log amplitude'); plt.imshow(log_amplitude)
plt.subplot(2, 3, 3); plt.title('amplitude histogram'); plot_hist(amplitude, 2000)
plt.subplot(2, 3, 4); plt.title('log amplitude histogram'); plot_hist(log_amplitude, 0.2)
plt.subplot(2, 3, 5); plt.title('phase'); plt.imshow(phase)
plt.subplot(2, 3, 6); plt.title('phase histogram'); plot_hist(phase, 0.085)

plt.show()

**b)** Display the amplitude and phase in separate images. You may again take the logarithm of
the amplitude to enhance the contrast. You may also center the base frequency (see function `numpy.fft.fftshift`). Compare your results with CV-09, slide 33.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('images/dolly.png')

# YOUR CODE HERE

# computes the n-dimensional discrete Fourier transform
ft = np.fft.fft2(img)
amplitude = np.abs(ft)
center_base_log_amplitude = np.fft.fftshift(np.log(amplitude))
phase = np.angle(ft)

# generate plots
plt.figure(figsize=(15, 10))
plt.gray()

plt.subplot(2, 2, 1); plt.title('original img'); plt.imshow(img)
plt.subplot(2, 2, 2); plt.title('phase img'); plt.imshow(phase)
plt.subplot(2, 2, 3); plt.title('amplitude img'); plt.imshow(amplitude)
plt.subplot(2, 2, 4); plt.title('center base log amplitude'); plt.imshow(center_base_log_amplitude)

plt.show()

**c)** Transform the image back from the frequency space to the image space (again using `fft2`).
What do you observe? Explain and repair the result.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('images/dolly.png')

# YOUR CODE HERE

# computes the n-dimensional discrete Fourier transform
ft = np.fft.fft2(img)
# apply FT to the resulting FT img
restored = np.abs(np.fft.fft2(ft))

# generate plots
plt.figure(figsize=(15, 10))
plt.gray()

plt.subplot(1, 3, 1); plt.title('original img'); plt.imshow(img)
plt.subplot(1, 3, 2); plt.title('restored'); plt.imshow(restored)
plt.subplot(1, 3, 3); plt.title('restored + repaired'); plt.imshow(np.fliplr(np.flipud(restored)))

plt.show()

One 2D-DFT property is that if you apply it twice, you end up with the original signal flipped (horizontally and vertically).  
We fix it by flipping it back horizontally and vertically.

**d)** Now restore the image, but only keep the amplitude and vary the phase. Try fixed phase
values (0, $\pi/2$,. . . ), a random phase matrix, or a noisy version of the original phase values.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('images/dollySquared.jpg')

# YOUR CODE HERE
ft = np.fft.fft2(img)
amplitude = np.abs(ft)
phase = np.angle(ft)

phases = []
phases.append((phase, "original phase"))
phases.append((np.zeros(phase.shape), "zero phase"))
phases.append((np.ones(phase.shape) * np.pi / 2, "pi/2 phase"))
phases.append((np.random.rand(*phase.shape) * 2 * np.pi, "random phase"))
phases.append((phase + np.random.rand(*phase.shape) * 2, "phase + noise"))

plt.figure(figsize=(12, 16)); plt.gray()
plt.subplot(3, 2, 1); plt.axis('off'); plt.imshow(img); plt.title('original img')

for i, (phase_val, title) in enumerate(phases):
    ft = amplitude * np.exp(1j * phase_val)
    img = np.abs(np.fft.ifft2(ft))
    plt.subplot(3, 2, 2+i); plt.axis('off'); plt.title(title)
    plt.imshow(img)

plt.show()

**e)** We do the same, but now we keep the phase while varying the amplitude values, i.e. constant,
amplitude, randomly distributed amplitudes and noisy versions of the the original values.

Explain the results!


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread('images/dolly.png')

# Perform (fast) Fourier transform
ft = np.fft.fft2(img)

# Get amplitude and phase
amplitude = np.abs(ft)
phase = np.angle(ft)

amplitudes = []
amplitudes.append((amplitude, "original amplitude"))
amplitudes.append((np.zeros(amplitude.shape), "zero amplitude"))
amplitudes.append((np.ones(amplitude.shape) * amplitude.max(), "constant amplitude"))
amplitudes.append((np.random.rand(*amplitude.shape), "random amplitude"))
amplitudes.append((amplitude, "original amplitude"))

plt.figure(figsize=(12, 16)); plt.gray()
plt.subplot(3, 2, 1); plt.axis('off'); plt.imshow(img); plt.title('Original')

for i, (amp, tit) in enumerate(amplitudes):
    ft0 = amp * np.exp(1j * phase)
    img0 = np.abs(np.fft.ifft2(ft0))
    plt.subplot(3, 2, 2+i); plt.axis('off'); plt.title(tit)
    plt.imshow(img0)

plt.show()

The phase is more important than the amplitude; destroying the phase would destroy all structure in the image.  
The amplitude controls the overall brightness in the image. Since we have the true phase in all cases,  
we see the structure of the image in each case for which the amplitude leads to an image that is bright enough to recognize the structure.

- img2 and img6: original phase + original amplitude --> perfect reconstruction
- img3: zero everywhere --> only black pixels
- img4 and img5: amplitude leads to high enough brightness to recognize the structure

## Exercise 2: Implementing Fourier Transform [6 points]

**a)** 
Explain in your own words the idea of Fourier transform. What is the frequency space? What does a point in that space represent?

The **Fourier transform** is a mathematical tool that transforms (global operation) the given information (image) into another domain.  
In our case, using it for images, we transform the information from the spatial domain into the frequency domain.  
Sometimes the frequency space enables more efficient computations, e.g. for convolutions.

So, the Fourier transform transforms the signal into the frequency space, where it is a sum (or integral) of sine waves of different frequencies,
each of which represents a frequency component.  
A point in that space would be a combination of weighted functions (sin / cos curves).


**b)** First implement a one-dimensional discrete version of Fourier transform, i.e. use the formula
$ c_n = \sum_{x=0}^{L-1} f(x)\cdot e^{-\tfrac{2\pi i\cdot n}{L}\cdot x} \qquad$ for $n=0,\ldots,L-1$
for complex valued coefficients.

Plot the graph and the results of your Fourier transform, using the Matplotlib function `plot()`, for different functions. Compare your results with the output of the function `numpy.fft.fft`.

In [ ]:
%matplotlib inline
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt


def fourier1d(func):
    """
    Perform a discrete 1D Fourier transform.
    
    Args:
        func (ndarray): 1-D array containing the function values.
    
    Returns:
        ndarray (complex): The Fourier transformed function.
    """
    ft = np.zeros(func.shape, dtype=np.complex)

    # YOUR CODE HERE

    L = len(func)
    
    for n in range(L):
        s = 0
        for x in range(L):
            s += func[x] * np.exp(-2 * np.pi * 1j * n * x / L)
        ft[n] = s
    return ft


# number of points
L = np.arange(100)


def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


func = np.sin(2 * np.pi * L / len(L))
#func = np.zeros(L.shape)
#func[40:60] = 1
#func = gaussian(L, 0, 10)

# Own implementation.
ft = fourier1d(func)
plt.figure(figsize=(12, 8))
plt.suptitle('Own implementation');
plt.subplot(1, 3, 1); plt.plot(L, func); plt.title('Function')
plt.subplot(1, 3, 2); plt.plot(L, np.abs(ft)); plt.title('FT (Amplitude)')
plt.subplot(1, 3, 3); plt.plot(L, np.angle(ft)); plt.title('FT (Phase)')
plt.show()

# Numpy implementation.
ft = np.fft.fft(func)

plt.figure(figsize=(12, 8))
plt.suptitle('Numpy implementation (FFT)')
plt.subplot(1, 3, 1); plt.plot(L, func); plt.title('Function')
plt.subplot(1, 3, 2); plt.plot(L, np.abs(ft)); plt.title('FT (Amplitude)')
plt.subplot(1, 3, 3); plt.plot(L, np.angle(ft)); plt.title('FT (Phase)')
plt.show()

**c)** Now implement a 2-dimensional version of Fourier transform for images, using the formula from the lecture. Compare your result with the output of `fft2`.  
![](images/FT.png)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


def fourier2d(img):
    """
    Perform discrete 2D Fourier transform of a given image.
    
    Args:
        img (ndarray): Input image.
        
    Returns:
    
    """
    ft = np.zeros(img.shape, dtype=np.complex)

    # YOUR CODE HERE
    M, N = img.shape
    x, y = np.meshgrid(np.arange(M), np.arange(N))

    for u in range(M):
        for v in range(N):
            s = 0
            # too slow - but that's the idea:
            # for x in range(M):
            #    for y in range(N):
            #        s += img[x][y] * np.exp(-1j * 2 * np.pi * (u * x / M + v * y / N))
            # ft[u][v] = s

            # more efficient - using meshgrid to create a rectangular grid out of an array 
            # of x values and an array of y values
            ft[u][v] = np.sum(img[x, y] * np.exp(-1j * 2 * np.pi * (u * x / M + v * y / N)))

    return ft


# Now (visually) compare your results with np.fft.fft2
img = plt.imread('images/dolly.png')[250:400, 250:400]

# YOUR CODE HERE

ft = np.fft.fft2(img)
log_amplitude = np.log(np.abs(ft))
phase = np.angle(ft)
reconstruction = np.real(np.fft.ifft2(ft))

own_ft = fourier2d(img)
own_log_amplitude = np.log(np.abs(own_ft))
own_phase = np.angle(own_ft)
own_reconstruction = np.real(np.fft.ifft2(own_ft))

plt.figure(figsize=(14, 8))
plt.gray()

plt.subplot(2, 3, 1); plt.title('fft2 phase'); plt.imshow(phase)
plt.subplot(2, 3, 2); plt.title('fft2 log amplitude'); plt.imshow(log_amplitude)
plt.subplot(2, 3, 3); plt.title('fft2 reconstruction'); plt.imshow(reconstruction)
plt.subplot(2, 3, 4); plt.title('own phase'); plt.imshow(own_phase)
plt.subplot(2, 3, 5); plt.title('own amplitude'); plt.imshow(own_log_amplitude)
plt.subplot(2, 3, 6); plt.title('own reconstruction'); plt.imshow(own_reconstruction)

plt.show()

## Exercise 3: Convolution theorem [7 points]

**a)** What does the convolution theorem state and what are its practical consequences?

The **convolution theorem** states that under suitable conditions the Fourier transform of a convolution of two signals
is the pointwise product of their Fourier transforms.  
Put simply, a **convolution in the spatial domain** corresponds to a **multiplication in Fourier space** (i.e. frequency domain).  
The practical consequence is that if the kernel used for a convolution is very large, it may be more efficient to perform the corresponding multiplication in Fourier space.

**b)**
When introducing convolution, we have discussed different methods to deal with boundary pixels. From the perspective of Fourier analysis, what is the natural way to deal with this problem?

A natural way to handle the boundary problem would be a **periodic extension** of the image.

**c)** What is the complexity for computing a convolution using the convolution theorem? Compare this with your complexity results from sheet 01, Assignment 1(c).


$n \times n$ img, $m \times m$ kernel  

**Using convolution theorem - $4$ steps:**
- FT of image: $O(n^2 \log n)$
- FT of kernel: $O(m^2 \log m)$
- multiplication in FT space: $O(n^2)$
- transform back to image space: $O(n^2 \log n)$  

$\rightarrow O(n^2 \log n)$

**In the image space (not using convolution theorem):**  

$O(m^2)$ for each of the $n^2$ pixels and therefore $O(n^2 \cdot m^2)$

Therefore, using the convolution theorem is worthwhile for big kernel sizes.

**d)** Proof the convolution theorem.

YOUR ANSWER HERE

## Exercise 4: Applying Fourier Transform [Bonus]

If you solve this exercise you may leave out one of the other exercises.


**a)** In order to apply the Custom Structuring Element to our satelite image in Assignment 4 of Sheet 3 we had to rotate the image. We had to measure the rotation angle by hand. We can now do this automatically via Fourier Transform.

1. Apply Fourier transform to the `img_gray`. The resulting amplitude should show the angle of the black lines.

1. Try to automatically get the rotation angle from the Fourier space. There are different ways to achieve this.
   Hints:
   * You may threshold the amplitudes, to only keep “relevant” values. You can then compute the angle of the largest relevant value.
   * Alternatively, you may apply methods you know from other lectures to get the main component and compute its angle.

1. Rotate the image back to its originally intended orientation (`skimage.transform.rotate`).

In [ ]:
%matplotlib inline
import numpy as np
from skimage import color
from skimage.transform import hough_line
from skimage.transform import rotate
import matplotlib.pyplot as plt

img = plt.imread('images/landsat_stack2.png')
img_gray = color.rgb2gray(img)

# YOUR CODE HERE

plt.show()

**b)** Can you think of other applications of Fourier Transform in Computer Vision?

YOUR ANSWER HERE